# 0. Importing the Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. Getting Tickers and Tweets

In [3]:
files = []
for file in os.listdir('data'):
    if file.endswith('.csv'):
        files.append(os.path.join('data', file))

In [4]:
data = []

for filename in files:
    with open(filename, 'r') as file:
        ticker = filename.split('_')[-1][:-4]
        line = file.readline()
        line = file.readline()
        while(line):
            ts =line.split(';"')[0][1:].split(';')[0]
            tweet = line.split(';"')[1].split('";')[0]
            line=file.readline()
            data.append([ticker,ts,tweet])
        
df = pd.DataFrame(data, columns=['ticker','timestamp','tweet'])
#df

In [5]:
df.head(10)

,ticker,timestamp,tweet
0,INTC,2019-03-21 23:29,$ AAPL $ TRV $ MSFT $ HD $ DWDP $ INTC $ NKE $...
1,INTC,2019-03-21 23:25,$ INTC Intel - Q3 2019 Intel Corporation Earni...
2,INTC,2019-03-21 23:25,$ INTC # Intel # Intelligence # ImplementingCl...
3,INTC,2019-03-21 23:25,$ INTC Intel - Q2 2019 Intel Corporation Earni...
4,INTC,2019-03-21 23:24,$ INTC Intel - Q1 2019 Intel Corporation Earni...
5,INTC,2019-03-21 23:20,Intel Corporation $ INTC Analysts See $0.87 EP...
6,INTC,2019-03-21 23:20,"On April, 25 The EPS for Intel Corporation $ I..."
7,INTC,2019-03-21 23:19,$0.87 EPS Expected for Intel Corporation $ INT...
8,INTC,2019-03-21 22:42,Apple Debuts a Host of Product Upgrades -- Wit...
9,INTC,2019-03-21 22:27,$ AAPL $ COTY $ AMD $ GE $ MO $ FB $ BAC $ NVD...


In [6]:
df['timestamp'] = pd.to_datetime(df['timestamp'], infer_datetime_format=True)

# 2. Adding Sentiment Score to Each Tweet

In [8]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [9]:
analyzer = SentimentIntensityAnalyzer()
for tweet in df['tweet']:
    vs = analyzer.polarity_scores(tweet)
    vs_val = list(vs.values())
    df['neg'] = vs_val[0]
    df['neu'] = vs_val[1]
    df['pos'] = vs_val[2]
    df['compound'] = vs_val[3]

In [10]:
df['day_name'] = df['timestamp'].map(lambda x: x.day_name())
df['day'] = df['timestamp'].map(lambda x: x.day)
df['day_of_week'] = df['timestamp'].map(lambda x: x.dayofweek)

In [53]:
#df.to_csv('tweets_with_sentiment.csv')

In [11]:
df.head(10)

,ticker,timestamp,tweet,neg,neu,pos,compound,day_name,day,day_of_week
0,INTC,2019-03-21 23:29:00,$ AAPL $ TRV $ MSFT $ HD $ DWDP $ INTC $ NKE $...,0.046,0.954,0.0,-0.3034,Thursday,21,3
1,INTC,2019-03-21 23:25:00,$ INTC Intel - Q3 2019 Intel Corporation Earni...,0.046,0.954,0.0,-0.3034,Thursday,21,3
2,INTC,2019-03-21 23:25:00,$ INTC # Intel # Intelligence # ImplementingCl...,0.046,0.954,0.0,-0.3034,Thursday,21,3
3,INTC,2019-03-21 23:25:00,$ INTC Intel - Q2 2019 Intel Corporation Earni...,0.046,0.954,0.0,-0.3034,Thursday,21,3
4,INTC,2019-03-21 23:24:00,$ INTC Intel - Q1 2019 Intel Corporation Earni...,0.046,0.954,0.0,-0.3034,Thursday,21,3
5,INTC,2019-03-21 23:20:00,Intel Corporation $ INTC Analysts See $0.87 EP...,0.046,0.954,0.0,-0.3034,Thursday,21,3
6,INTC,2019-03-21 23:20:00,"On April, 25 The EPS for Intel Corporation $ I...",0.046,0.954,0.0,-0.3034,Thursday,21,3
7,INTC,2019-03-21 23:19:00,$0.87 EPS Expected for Intel Corporation $ INT...,0.046,0.954,0.0,-0.3034,Thursday,21,3
8,INTC,2019-03-21 22:42:00,Apple Debuts a Host of Product Upgrades -- Wit...,0.046,0.954,0.0,-0.3034,Thursday,21,3
9,INTC,2019-03-21 22:27:00,$ AAPL $ COTY $ AMD $ GE $ MO $ FB $ BAC $ NVD...,0.046,0.954,0.0,-0.3034,Thursday,21,3


# 3. Getting Stock Data

## 3.1 Aggregate Stock Data (S&P500 Top 50)

In [12]:
agg_sp50 = pd.read_excel('S&PTop50 Aggregate Data.xls')

In [13]:
agg_sp50.head()

,Effective date,S&P 500 Top 50 TR,S&P 500 Top 50 NTR,S&P 500 Top 50
0,2019-03-01,3053.12,2771.73,2211.81
1,2019-03-04,3048.79,2767.80,2208.68
2,2019-03-05,3050.96,2769.77,2210.25
3,2019-03-06,3036.45,2756.60,2199.74
4,2019-03-07,3010.50,2733.04,2180.94


## 3.2 Individual Stock Data (S&P500 Top 50)

In [17]:
len(df['ticker'].unique())

49

In [18]:
tickers = list(set(df['ticker'].unique()))

In [19]:
tickers.append('BRK-B')
tickers.remove('BRK')

In [20]:
len(tickers)

49

In [27]:
import fix_yahoo_finance as yf

complete_panel_data = yf.download(tickers, start="2019-03-01", end="2019-03-29")

[*********************100%***********************]  49 of 49 downloaded


In [32]:
complete_panel_data.head()

Open                                                \
                  AAPL       ABBV        ABT        ADBE        AMGN   
Date                                                                   
2019-03-01  174.279999  79.769997  77.989998  265.750000  191.389999   
2019-03-04  175.690002  80.400002  78.839996  266.700012  191.820007   
2019-03-05  175.940002  79.379997  78.559998  259.000000  190.000000   
2019-03-06  174.669998  79.129997  78.110001  257.399994  188.880005   
2019-03-07  173.869995  77.769997  77.690002  255.770004  182.919998   

                                                                      ...     \
                   AMZN        AVGO          BA    BAC       BRK-B    ...      
Date                                                                  ...      
2019-03-01  1655.130005  277.730011  446.010010  29.33  203.149994    ...      
2019-03-04  1685.000000  275.000000  443.220001  29.33  203.089996    ...      
2019-03-05  1702.949951  275.500000  431.040009  29.02  201.759995    ...      
2019-03-06  1695.969971  276.660004  429.700012  29.01  201.610001    ...      
2019-03-07  1667.369995  270.179993  423.359985  28.67  200.580002    ...      

             Volume                                                          \
                 PM     PYPL         T      UNH      UNP        V        VZ   
Date                                                                          
2019-03-01  4379100  5434000  30592900  6921300  3116100  7016800  10942700   
2019-03-04  3711700  7855800  49482700  8646300  4341000  6979700  13674400   
2019-03-05  5547100  6321900  26029800  6924500  2794900  9678600  15749500   
2019-03-06  4938400  3226500  24580300  4878700  3067800  8186400  13935500   
2019-03-07  4802400  6170100  24826100  5033500  4141500  8761700  12644200   

                                          
                 WFC       WMT       XOM  
Date                                      
2019-03-01  19051400  10352500  15419400  
2019-03-04  16697000   7683800  18089900  
2019-03-05  14235400   6119300  13745500  
2019-03-06  14211000   4696600  18633600  
2019-03-07  19047300   6126600  16541700  

[5 rows x 294 columns]

In [38]:
complete_panel_data.loc[complete_panel_data.index]['Adj Close']

,AAPL,ABBV,ABT,ADBE,AMGN,AMZN,AVGO,BA,BAC,BRK-B,...,PM,PYPL,T,UNH,UNP,V,VZ,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-01,174.970001,80.099998,78.589996,264.010010,191.160004,1671.729980,270.335175,440.619995,29.309999,203.130005,...,86.415771,98.800003,30.820000,245.214050,167.440002,149.470001,56.959999,50.029999,97.405891,80.000000
2019-03-04,175.850006,79.459999,78.620003,258.160004,189.779999,1696.170044,272.456238,432.690002,29.030001,201.809998,...,86.188644,96.570000,29.980000,235.122589,167.369995,147.960007,56.240002,50.110001,97.326317,80.309998
2019-03-05,175.529999,78.910004,78.220001,257.380005,188.820007,1692.430054,274.557465,430.119995,29.080000,201.470001,...,85.892395,96.529999,29.950001,240.512009,166.779999,147.949997,56.080002,49.889999,97.813690,80.190002
2019-03-06,174.520004,78.089996,77.699997,256.399994,183.149994,1668.949951,268.947571,424.459991,28.840000,200.669998,...,85.931892,96.459999,29.809999,238.160995,165.910004,147.809998,55.680000,49.820000,97.734123,79.279999
2019-03-07,172.500000,78.239998,76.800003,255.460007,181.460007,1625.949951,263.159271,422.559998,28.549999,199.300003,...,85.882523,96.209999,29.920000,235.799988,164.940002,146.830002,56.299999,49.680000,96.928452,80.160004
2019-03-08,172.910004,77.580002,76.629997,254.740005,180.869995,1620.800049,261.850952,422.540009,28.650000,198.699997,...,86.356514,95.690002,29.959999,237.289993,164.300003,147.350006,56.529999,49.799999,97.067703,79.010002
2019-03-11,178.899994,78.080002,77.809998,260.959991,184.139999,1670.619995,266.677826,400.010010,28.940001,202.089996,...,88.124130,97.449997,30.219999,243.149994,165.600006,150.669998,57.580002,49.759998,97.952950,79.779999
2019-03-12,180.910004,78.739998,78.239998,263.510010,184.179993,1673.099976,267.242798,375.410004,28.940001,202.500000,...,87.689636,97.500000,30.629999,245.880005,164.830002,151.729996,57.430000,49.650002,97.843536,80.000000
2019-03-13,181.710007,78.930000,78.620003,264.380005,187.350006,1690.810059,268.808777,377.140015,29.290001,204.059998,...,88.380882,99.309998,30.280001,252.250000,166.830002,152.470001,57.660000,49.919998,98.500000,80.709999


In [23]:
# import pandas_datareader

# start_date = '2019-03-01'
# end_date = '2019-03-26'

# complete_panel_data = pandas_datareader.get_data_yahoo(tickers, start_date, end_date)